# Homework

##### Dataset

In this homework, we will use the Bank Marketing dataset (https://archive.ics.uci.edu/static/public/222/bank+marketing.zip).

In this dataset our desired target for classification task will be y variable - has the client subscribed a term deposit or not.

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from io import BytesIO
import urllib.request as urllib2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score


In [14]:
# loading the temp.zip and creating a zip object 
with ZipFile("C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\bank+marketing.zip", 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zObject.extractall(path="C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\temp") 

with ZipFile("C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\temp\\bank.zip", 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zObject.extractall(path="C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\temp") 

df = pd.read_csv('C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\temp\\bank.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


##### Features
For the rest of the homework, you'll need to use only these columns:

1) age,
2) job,
3) marital,
4) education,
5) balance,
6) housing,
7) contact,
8) day,
9) month,
10) duration,
11) campaign,
12) pdays,
13) previous,
14) poutcome,
15) y

##### Data preparation

Select only the features from above.

Check if the missing values are presented in the features.

In [15]:
base = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
df_num = df[base]

In [16]:
print("Columns with NaN: ", df_num.isnull().any()) 

Columns with NaN:  age          False
job          False
marital      False
education    False
balance      False
housing      False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool


#### Question 1

What is the most frequent observation (mode) for the column education?

1) unknown
2) primary
3) **secondary**
4) tertiary

In [17]:
mode = df_num.education.mode()
print(mode)

0    secondary
Name: education, dtype: object


#### Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

1) age and balance
2) day and campaign
3) day and pdays
4) **pdays and previous**

In [18]:
corrM = df_num.corr(numeric_only=True)
print(corrM)

               age   balance       day  duration  campaign     pdays  previous
age       1.000000  0.083820 -0.017853 -0.002367 -0.005148 -0.008894 -0.003511
balance   0.083820  1.000000 -0.008677 -0.015950 -0.009976  0.009437  0.026196
day      -0.017853 -0.008677  1.000000 -0.024629  0.160706 -0.094352 -0.059114
duration -0.002367 -0.015950 -0.024629  1.000000 -0.068382  0.010380  0.018080
campaign -0.005148 -0.009976  0.160706 -0.068382  1.000000 -0.093137 -0.067833
pdays    -0.008894  0.009437 -0.094352  0.010380 -0.093137  1.000000  0.577562
previous -0.003511  0.026196 -0.059114  0.018080 -0.067833  0.577562  1.000000


##### Target encoding

Now we want to encode the y variable.

Let's replace the values yes/no with 1/0.

In [19]:
df_num['y'] = df_num['y'].map({'yes': 1, 'no': 0})
#df_num = df_num.assign(above_average = df_num.y)
#df_num['above_average'] = np.where(df_num['y'] > df_num['y'].mean(), 1, 0) 
df_num

C:\Users\Sandra\AppData\Local\Temp\ipykernel_8412\1771879114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num['y'] = df_num['y'].map({'yes': 1, 'no': 0})


,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,1787,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,4789,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,1350,yes,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,1476,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,yes,unknown,5,may,226,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,-333,yes,cellular,30,jul,329,5,-1,0,unknown,0
4517,57,self-employed,married,tertiary,-3313,yes,unknown,9,may,153,1,-1,0,unknown,0
4518,57,technician,married,secondary,295,no,cellular,19,aug,151,11,-1,0,unknown,0
4519,28,blue-collar,married,secondary,1137,no,cellular,6,feb,129,4,211,3,other,0


##### Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value y is not in your dataframe.

In [20]:
df_full_train, df_test = train_test_split(df_num, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_val['y']
del df_test['y']


In [21]:
df_full_train = df_full_train.reset_index(drop=True)
df_full_train.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [22]:
df_full_train.y.value_counts(normalize=True)

y
0    0.88302
1    0.11698
Name: proportion, dtype: float64

In [23]:
df_full_train.y.mean()

np.float64(0.11698008849557522)

#### Question 3

Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.

Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

1) contact
2) education
3) housing
4) **poutcome**

In [24]:
categorical = ['job', 'marital', 'education','housing', 'contact', 'month', 'poutcome']

numerical = ['age', 'balance', 'day', 'duration','campaign', 'pdays', 'previous', 'y']

In [25]:
df_full_train[categorical].nunique()

job          12
marital       3
education     4
housing       2
contact       3
month        12
poutcome      4
dtype: int64

In [ ]:
def calculate_mi(series):
    return mutual_info_score(series, df_full_train.above_average)

df_mi = df_full_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

display(df_mi.head())

#### Question 4

Now let's train a logistic regression.

Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.

Fit the model on the training dataset.

To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

1) 0.6
2) 0.7
3) 0.8
4) **0.9**

In [27]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
#train_dict = df_train[categorical + numerical].to_dict(orient='records')
#X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
#val_dict = df_val[categorical + numerical].to_dict(orient='records')
#X_val = dv.transform(val_dict)

In [28]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [29]:
def logistic_regression(xi):
    score = w0
    
    for j in range(len(w)):
        score = score + xi[j] * w[j]
        
    result = sigmoid(score)
    return result

In [30]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

accuracy = np.round(accuracy_score(y_val, y_pred),2)
print(f'Accuracy = {accuracy}')

Accuracy = 0.89


#### Question 5

Let's find the least useful feature using the feature elimination technique.

Train a model with all these features (using the same parameters as in Q4).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

1) age
2) balance
3) **marital**
4) previous

Note: The difference doesn't have to be positive.

In [32]:
df_full_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,34,entrepreneur,divorced,tertiary,262,no,cellular,20,nov,371,1,-1,0,unknown,0
1,32,management,married,tertiary,2349,no,cellular,14,aug,134,5,-1,0,unknown,0
2,34,technician,single,secondary,1076,no,cellular,8,aug,70,2,-1,0,unknown,0
3,31,management,married,tertiary,156,no,cellular,13,aug,657,7,-1,0,unknown,1
4,46,blue-collar,married,primary,258,yes,unknown,27,may,217,1,-1,0,unknown,0


In [43]:
features = df_train.columns.to_list()
features


['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [46]:
original_score = accuracy
scores = pd.DataFrame(columns=['eliminated_feature', 'accuracy', 'difference'])
for feature in features:
    subset = features.copy()
    subset.remove(feature)
    
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[subset].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    val_dict = df_val[subset].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    
    scores.loc[len(scores)] = [feature, score, original_score - score]

    min_diff = scores.difference.min()
    scores[scores.difference == min_diff]

print(scores)

   eliminated_feature  accuracy  difference
0                 age  0.889503    0.000497
1                 job  0.893646   -0.003646
2             marital  0.892265   -0.002265
3           education  0.889503    0.000497
4             balance  0.888122    0.001878
5             housing  0.888122    0.001878
6             contact  0.883978    0.006022
7                 day  0.888122    0.001878
8               month  0.886740    0.003260
9            duration  0.871547    0.018453
10           campaign  0.882597    0.007403
11              pdays  0.888122    0.001878
12           previous  0.889503    0.000497
13           poutcome  0.877072    0.012928


#### Question 6

Now let's train a regularized logistic regression.

Let's try the following values of the parameter C: [0, 0.01, 0.1, 1, 10].

Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

1) 0.01
2) 0.1
3) 1
4) **10**
5) 100

Note: If there are multiple options, select the smallest C.

In [47]:
df = pd.read_csv('C:\\Users\\Sandra\\Documents\\Cursos\\MLZoomcamp2024\\Lesson_3\\Homework\\temp\\bank.csv', sep=';')

#Rename MSRP variable to y
df=df.rename(columns={"msrp": "y"})

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_val['y']
del df_test['y']


dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
scores = {}
for alpha in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', max_iter=1000, C=alpha, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    score = accuracy_score(y_val, y_pred)
    scores[alpha] = round(score, 3)
    print(f'alpha = {alpha}:\t RMSE = {score}')

print(scores)
print(f'The smallest `alpha` is {max(scores, key=scores.get)}.')


alpha = 0.01:	 RMSE = 0.8729281767955801
alpha = 0.1:	 RMSE = 0.888121546961326
alpha = 1:	 RMSE = 0.8922651933701657
alpha = 10:	 RMSE = 0.8964088397790055
alpha = 100:	 RMSE = 0.8922651933701657
{0.01: 0.873, 0.1: 0.888, 1: 0.892, 10: 0.896, 100: 0.892}
The smallest `alpha` is 10.
